In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Lastsemester/Social_Network_Analysis/project

Mounted at /content/gdrive
/content/gdrive/My Drive/Lastsemester/Social_Network_Analysis/project


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import pandas as pd
import numpy as np

d = {'product_id': ['B0188A3QRM', 'B075VXJ9VG'], 
     'product_title': ['Amazon Basics Woodcased #2 Pencils, Unsharpened', 'BAZIC Pencil #2 HB Pencils, Latex Free Eraser'],
     'product_description': [np.NaN, '<p><strong>BACK TO BAZIC</strong></p><p>Our go'], 
     'product_bullet_point': ['144 woodcase #2 HB pencils made from high-quality', '&#11088; UN-SHARPENED #2 PREMIUM PENCILS.'],
     'product_brand': ['Amazon Basics	', 'BAZIC Products	'],
     'product_color_name': [ 'Yellow', '12-count'],
     'product_locale': [ 'us', 'us']}
df = pd.DataFrame(data=d)

# add ".astype(str)" 
df['Combined'] = df['product_title'].astype(str) + ', ' + df['product_description'].astype(str) + ', ' + df['product_bullet_point'].astype(str)  + ', ' + df['product_brand'].astype(str) + ', ' + df['product_color_name'].astype(str) + ', ' + df['product_locale'].astype(str)
df = pd.DataFrame(data=df)
s = df['Combined']
df



# pass in the real data here, as "df"

In [42]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('wk7_lr').getOrCreate()

from pyspark.sql.types import StructType,StructField, StringType, IntegerType

schema = StructType([ \
    StructField("product_id",StringType(),True), \
    StructField("product_title",StringType(),True), \
    StructField("product_description",StringType(),True), \
    StructField("product_bullet_point", StringType(), True), \
    StructField("product_brand", StringType(), True), \
    StructField("product_color_name", StringType(), True), \
    StructField("product_locale", StringType(), True), \
    StructField("Combined", StringType(), True) \
  ])
sparkDF=spark.createDataFrame(df, schema = schema) 



In [31]:
sparkDF.show()

+----------+--------------------+--------------------+--------------------+----------------+------------------+--------------+--------------------+
|product_id|       product_title| product_description|product_bullet_point|   product_brand|product_color_name|product_locale|            Combined|
+----------+--------------------+--------------------+--------------------+----------------+------------------+--------------+--------------------+
|B0188A3QRM|Amazon Basics Woo...|                 NaN|144 woodcase #2 H...| Amazon Basics\t|            Yellow|            us|Amazon Basics Woo...|
|B075VXJ9VG|BAZIC Pencil #2 H...|<p><strong>BACK T...|&#11088; UN-SHARP...|BAZIC Products\t|          12-count|            us|BAZIC Pencil #2 H...|
+----------+--------------------+--------------------+--------------------+----------------+------------------+--------------+--------------------+



In [37]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

tokenizer = Tokenizer(inputCol="Combined", outputCol="words")
wordsData = tokenizer.transform(sparkDF)
wordsData.show(truncate=False)



+----------+-----------------------------------------------+----------------------------------------------+-------------------------------------------------+----------------+------------------+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                  |product_description                           |product_bullet_point                             |product_brand   |product_color_name|product_locale|Combined                                                                                                                                                                |words                                                 

In [41]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="words", outputCol="result")
model = word2Vec.fit(wordsData)

result = model.transform(wordsData)
result.show(truncate=False)


+----------+-----------------------------------------------+----------------------------------------------+-------------------------------------------------+----------------+------------------+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                  |product_description                           |product_bullet_point                             |product_brand   |product_color_name|product_locale